In [1]:
# This is a sample code showing how to link CRSP, Compustat and SEC Edgar data.
# It also assumes that task I is finished and 10-K filng dates and CIK numbers have been obtained.

import wrds    
import random
import numpy as np
import pandas as pd

# Create a connection to WRDS server
# You will be asked for your username and passwords

conn = wrds.Connection()


Enter your WRDS username [xuh14]:vaskatta
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [2]:
# As an example, we are going to focus on two firms, Apple and Microsoft, and their 10-K reports in 2016. 
# Their CIK numbers are 0000320193 and 0000789019, respectively; the filing dates are 2016-09-30 and 2016-06-30, respectively. 
# You should be able to obtain the CIK numbers and filing dates from task I.

# Computstat has a field called CIK. Therefore, we can find a firm's GVKEY, which is the Compustat's identification key, 
# by matching CIK in Compustat. In addition to GVKEY, we also try to obtain two other variables: book value per share, bkvlps,
# and the date on which the Compustat data was observed, datadate.


# we put all the CIK numbers in a tuple.
CIK_list=tuple([ "0000320193", "0000789019"]);
fd_list =tuple(["2016-10-26", "2016-07-28"]);

fyear=2016;

stmt1 = """
select gvkey, bkvlps, datadate, cik
from comp.funda
where  cik in {}
and fyear={}
""".format(CIK_list, fyear);

gvkey_list=conn.raw_sql(stmt1);
print(gvkey_list);



    gvkey   bkvlps    datadate         cik
0  001690  24.0339  2016-09-30  0000320193
1  001690      NaN  2016-09-30  0000320193
2  012141   9.2209  2016-06-30  0000789019
3  012141      NaN  2016-06-30  0000789019


In [3]:
# bkvlps is book value per share. As you can see, there might be missing values when querying data. We only keep the non-missing
# values.

gvkey_list = gvkey_list[np.isfinite(gvkey_list['bkvlps'])];
print(gvkey_list);

    gvkey   bkvlps    datadate         cik
0  001690  24.0339  2016-09-30  0000320193
2  012141   9.2209  2016-06-30  0000789019


In [4]:
# CRSP provides a linking table between CRSP and Compustat, showing the correspondence between PERMO and GVKEY. Therefore, we will
# use the GVKEY obtained from last step to find out their PERMOS in CRSP.

stmt2= """
select gvkey, lpermno as permno
from crsp.ccmxpf_linktable
where gvkey in {}
""".format(tuple(gvkey_list['gvkey']));

ccm= conn.raw_sql(stmt2)
print(ccm);



    gvkey   permno
0  001690      NaN
1  001690  14593.0
2  012141      NaN
3  012141  10107.0


In [5]:
ccm = ccm[np.isfinite(ccm['permno'])];
ccm['permno']=ccm['permno'].astype('int64', copy=False)
print(ccm);

    gvkey  permno
1  001690   14593
3  012141   10107


In [6]:
# Therfore, we find out the PERMNOS for Apple and Microsoft. So we can merge all information, 
# including GVKEY, CIK and PERMNO together.

ccm1=pd.merge(gvkey_list[['gvkey','datadate','bkvlps', 'cik']],ccm,how='left',on=['gvkey'])
print(ccm1);

    gvkey    datadate   bkvlps         cik  permno
0  001690  2016-09-30  24.0339  0000320193   14593
1  012141  2016-06-30   9.2209  0000789019   10107


In [9]:
# In principle, you can download all the stock return data from CRSP to local but that probably requires a great deal of 
# memeory and disk space. Alternatively, you can loop over each pair of filing date and CIK and find out the stock return, say,
# of 7 days before and after the filing date.

ccm2=pd.DataFrame();


for i in range(len(fd_list)):
    stmt3="""
    select permno, ret, retx, date(date) as date
    FROM crsp.dsf
    where permno= {}
    and  date < date '{}' + integer '7'
    and date > date '{}' - integer '7'
    """.format(ccm['permno'].values[i],fd_list[i],fd_list[i])
    ccm2=ccm2.append( conn.raw_sql(stmt3))
    
print(ccm2)

    permno       ret      retx        date
0  14593.0 -0.000512 -0.000512  2016-10-20
1  14593.0 -0.003930 -0.003930  2016-10-21
2  14593.0  0.009005  0.009005  2016-10-24
3  14593.0  0.005100  0.005100  2016-10-25
4  14593.0 -0.022495 -0.022495  2016-10-26
5  14593.0 -0.009603 -0.009603  2016-10-27
6  14593.0 -0.006639 -0.006639  2016-10-28
7  14593.0 -0.001583 -0.001583  2016-10-31
8  14593.0 -0.018055 -0.018055  2016-11-01
0  10107.0  0.013799  0.013799  2016-07-22
1  10107.0  0.002828  0.002828  2016-07-25
2  10107.0  0.000529  0.000529  2016-07-26
3  10107.0 -0.010042 -0.010042  2016-07-27
4  10107.0  0.000356  0.000356  2016-07-28
5  10107.0  0.008362  0.008362  2016-07-29
6  10107.0 -0.001764 -0.001764  2016-08-01
7  10107.0  0.000000  0.000000  2016-08-02
8  10107.0  0.006893  0.006893  2016-08-03


In [8]:
# Finally,we merge stock return data CCM2 to firm's characteristics, such as bkvlps

ccm3=pd.merge(ccm1, ccm2, how="right", on=['permno']);
print(ccm3)

     gvkey    datadate   bkvlps         cik  permno       ret      retx  \
0   001690  2016-09-30  24.0339  0000320193   14593 -0.000512 -0.000512   
1   001690  2016-09-30  24.0339  0000320193   14593 -0.003930 -0.003930   
2   001690  2016-09-30  24.0339  0000320193   14593  0.009005  0.009005   
3   001690  2016-09-30  24.0339  0000320193   14593  0.005100  0.005100   
4   001690  2016-09-30  24.0339  0000320193   14593 -0.022495 -0.022495   
5   001690  2016-09-30  24.0339  0000320193   14593 -0.009603 -0.009603   
6   001690  2016-09-30  24.0339  0000320193   14593 -0.006639 -0.006639   
7   001690  2016-09-30  24.0339  0000320193   14593 -0.001583 -0.001583   
8   001690  2016-09-30  24.0339  0000320193   14593 -0.018055 -0.018055   
9   012141  2016-06-30   9.2209  0000789019   10107  0.013799  0.013799   
10  012141  2016-06-30   9.2209  0000789019   10107  0.002828  0.002828   
11  012141  2016-06-30   9.2209  0000789019   10107  0.000529  0.000529   
12  012141  2016-06-30   